In [ ]:
!pip -q install pandas geopandas folium shapely matplotlib


In [ ]:
from google.colab import files
uploaded = files.upload()
LOGO_FILE = next(iter(uploaded.keys()))
LOGO_FILE


Saving PM SHRI SYMBOL.jpg to PM SHRI SYMBOL.jpg


'PM SHRI SYMBOL.jpg'

In [ ]:
from google.colab import files
print("Upload your Excel (school_dataset_new.xlsx) and bihar_districts.geojson")
up = files.upload()

Upload your Excel (school_dataset_new.xlsx) and bihar_districts.geojson


Saving school_dataset_new.xlsx to school_dataset_new.xlsx


In [ ]:
from google.colab import files
print("Upload your Excel (school_dataset_new.xlsx) and bihar_districts.geojson")
up = files.upload()

Upload your Excel (school_dataset_new.xlsx) and bihar_districts.geojson


Saving bihar_districts.geojson to bihar_districts.geojson


In [ ]:
import pandas as pd

DF_FILE = "school_dataset_new.xlsx"
df = pd.read_excel(DF_FILE)
df = df.rename(columns={
    "NAME OF SCHOOL": "school",
    "BLOCK NAME": "block",
    "NAME OF DISTRICTS": "district",
    "latitude": "latitude",
    "longitude": "longitude",
})
df = df.dropna(subset=["latitude","longitude"]).copy()
print("Rows to plot:", len(df))
df.head(3)


Rows to plot: 836


,school,block,district,latitude,longitude
0,HIGH SCHOOL SIMARBANI,BHARGAMA,ARARIA,26.191280,87.179010
1,RAJKIYAKRIT HIGH SCHOOL KUARI,KURSAKANTA,ARARIA,26.411400,87.442000
2,HIGH SCHOOL JOKIHAT,Araria (M),ARARIA,26.143511,87.619409


In [ ]:
import geopandas as gpd

GJ_FILE = "bihar_districts.geojson"
bihar = gpd.read_file(GJ_FILE).to_crs(epsg=4326)

bihar = bihar[bihar.geometry.type.isin(["Polygon","MultiPolygon"])].copy()

for c in bihar.columns:
    if c != "geometry" and str(getattr(bihar[c], "dtype", "")).startswith("geometry"):
        bihar = bihar.drop(columns=[c])
name_field = None
lower_districts = {d.strip().lower() for d in df["district"].astype(str).unique()}
for c in bihar.columns:
    if c == "geometry":
        continue
    vals = {str(v).strip().lower() for v in bihar[c].astype(str).unique()}
    overlap = len(lower_districts & vals)
    if overlap >= max(5, len(lower_districts)//4):
        name_field = c
        break


if name_field is None:
    for c in bihar.columns:
        if c.lower() in {"name", "district", "dist_name", "shapename"}:
            name_field = c; break
if name_field is None:
    name_field = [c for c in bihar.columns if c != "geometry"][0]


bihar = bihar[[name_field, "geometry"]].copy()
print("Using district name field:", name_field)
bihar.head(3)


Using district name field: NAME_2


,NAME_2,geometry
0,Araria,"MULTIPOLYGON (((87.671 26.1109, 87.6687 26.107..."
1,Arwal,"MULTIPOLYGON (((84.7315 25.0579, 84.7329 25.06..."
2,Aurangabad,"MULTIPOLYGON (((84.4402 25.1296, 84.4678 25.11..."


In [ ]:
import folium
from folium.map import CustomPane
from folium import GeoJson, GeoJsonTooltip


minx, miny, maxx, maxy = bihar.total_bounds

m = folium.Map(
    location=[(miny+maxy)/2, (minx+maxx)/2],
    zoom_start=7,
    tiles="CartoDB Positron",
    control_scale=True,
    prefer_canvas=True
)


CustomPane("district-borders", z_index=400).add_to(m)
CustomPane("school-markers",  z_index=650).add_to(m)

GeoJson(
    data=bihar.__geo_interface__,
    name="Bihar District Borders",
    pane="district-borders",
    style_function=lambda f: {"fillColor":"transparent","color":"#000","weight":2.8},
    tooltip=GeoJsonTooltip(fields=[name_field], aliases=["District:"], sticky=True),
).add_to(m)

m.fit_bounds([[miny, minx], [maxy, maxx]])
m.options["maxBounds"] = [[miny-0.3, minx-0.3], [maxy+0.3, maxx+0.3]]


In [ ]:
from folium import CustomIcon, Marker


icon = CustomIcon("PM SHRI SYMBOL.jpg", icon_size=(5, 5))

for _, r in df.iterrows():
    tooltip_txt = f"{r['school']} ({r['district']})"
    popup_html = f"""
    <div style="background:#3b5bdb;color:#fff;padding:8px 10px;border-radius:6px;
                font-family:Arial, sans-serif; box-shadow:0 2px 8px rgba(0,0,0,0.25);">
      <div style="font-weight:700;margin-bottom:6px">{r['school']}</div>
      <div>district: {r['district']}</div>
      <div>latitude: {r['latitude']:.5f}</div>
      <div>longitude: {r['longitude']:.5f}</div>
      <div>block: {r['block']}</div>
    </div>
    """
    Marker(
        location=[r["latitude"], r["longitude"]],
        tooltip=tooltip_txt,
        popup=folium.Popup(popup_html, max_width=320),
        icon=icon,
        pane="school-markers",
    ).add_to(m)


In [ ]:
OUT = "bihar_schools_logo_markers.html"
m.save(OUT)
print("Saved:", OUT)

from google.colab import files
files.download(OUT)


Saved: bihar_schools_logo_markers.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import geopandas as gpd

DF_FILE  = "school_dataset_new.xlsx"
GJ_FILE  = "bihar_districts.geojson"
LOGO_FILE = "PM_SHRI_SYMBOL.jpg"

# Schools
df = pd.read_excel(DF_FILE).rename(columns={
    "NAME OF SCHOOL": "school",
    "BLOCK NAME": "block",
    "NAME OF DISTRICTS": "district",
    "latitude": "latitude",
    "longitude": "longitude",
}).dropna(subset=["latitude","longitude"]).copy()

# District polygons
bihar = gpd.read_file(GJ_FILE).to_crs(epsg=4326)
bihar = bihar[bihar.geometry.type.isin(["Polygon","MultiPolygon"])].copy()
for c in bihar.columns:
    if c != "geometry" and str(getattr(bihar[c], "dtype", "")).startswith("geometry"):
        bihar = bihar.drop(columns=[c])

# Try to find the district name column in the GeoJSON
name_field = None
cand = {"name","district","dist_name","distname","shapename"}
for c in bihar.columns:
    if c != "geometry" and c.lower() in cand:
        name_field = c; break
if name_field is None:

    name_field = [c for c in bihar.columns if c != "geometry"][0]

print("Using district-name field in GeoJSON:", name_field)


Using district-name field in GeoJSON: GID_2


In [ ]:
print(bihar.columns)

Index(['GID_2', 'GID_0', 'COUNTRY', 'GID_1', 'NAME_1', 'NL_NAME_1', 'NAME_2',
       'VARNAME_2', 'NL_NAME_2', 'TYPE_2', 'ENGTYPE_2', 'CC_2', 'HASC_2',
       'geometry'],
      dtype='object')


In [ ]:
import folium
from folium import GeoJson, GeoJsonTooltip, Marker, DivIcon, CustomIcon
import matplotlib.cm as cm
import matplotlib.colors as colors

name_field = "NAME_2"
keep_cols = [name_field, "geometry"]
bihar_gj = bihar[keep_cols].copy()

#District color map based on the polygon layer to avoid name mismatches
districts_poly = sorted(bihar_gj[name_field].astype(str).unique())
cmap = cm.get_cmap("tab20", len(districts_poly))
poly_color = {d: colors.to_hex(cmap(i)) for i, d in enumerate(districts_poly)}

def style_fn(feat):
    dname = str(feat["properties"].get(name_field, ""))
    return {
        "fillColor": poly_color.get(dname, "#dcdcdc"),
        "color": "#000",
        "weight": 2.8,
        "fillOpacity": 0.30,
    }

#Build_map
minx, miny, maxx, maxy = bihar_gj.total_bounds
m = folium.Map(location=[(miny+maxy)/2, (minx+maxx)/2], zoom_start=7,
               tiles="CartoDB Positron", control_scale=True, prefer_canvas=True)

gj = GeoJson(
    data=bihar_gj.__geo_interface__,
    name="Bihar Districts",
    style_function=style_fn,
    tooltip=GeoJsonTooltip(fields=[name_field], aliases=["District:"], sticky=True),
)
gj.add_to(m)

m.fit_bounds([[miny, minx], [maxy, maxx]])
m.options["maxBounds"] = [[miny-0.3, minx-0.3], [maxy+0.3, maxx+0.3]]

#Bold labels at district centroids
label_points = bihar_gj.geometry.representative_point()
for pt, dname in zip(label_points, bihar_gj[name_field].astype(str)):
    Marker(
        location=[pt.y, pt.x],
        icon=DivIcon(html=f"""
            <div style="font-weight:800; font-size:12px; color:#111; text-align:center;
                        text-shadow:-1px -1px 0 #fff, 1px -1px 0 #fff,
                                     -1px  1px 0 #fff, 1px  1px 0 #fff;">
                {dname.upper()}
            </div>
        """)
    ).add_to(m)

icon = CustomIcon("PM SHRI SYMBOL.jpg", icon_size=(14, 14))
for _, r in df.iterrows():
    Marker(
        location=[r["latitude"], r["longitude"]],
        tooltip=f"{r['school']} ({r['district']})",
        icon=icon
    ).add_to(m)

legend = """
<div style="
  position: fixed; top: 20px; right: 20px; z-index: 9999;
  width: 240px; max-height: 70vh; overflow-y: auto;
  background: white; border: 2px solid #888; border-radius: 8px;
  font: 12px/1.3 Arial, sans-serif; padding: 8px 10px;">
  <div style="font-weight:700; margin-bottom:6px;">District Colors</div>
"""
for d in districts_poly:
    c = poly_color[d]
    legend += f"""
    <div style="display:flex; align-items:center; margin:3px 0;">
      <span style="display:inline-block; width:14px; height:14px; background:{c};
                   border:1px solid #333; margin-right:6px;"></span>
      <span>{d}</span>
    </div>"""
legend += "</div>"
m.get_root().html.add_child(folium.Element(legend))
#Save & download
OUT = "bihar_schools_filled_districts_labels_legend.html"
m.save(OUT)
print("Saved:", OUT)

from google.colab import files
files.download(OUT)


/tmp/ipython-input-121047335.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("tab20", len(districts_poly))


Saved: bihar_schools_filled_districts_labels_legend.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>